In [1]:
using ProgressMeter
using Random #, Distributions
using CUDA
;

In [5]:
function mock_energy_CUDA!(dim::Int,x, energies)
    index = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    stride = blockDim().x * gridDim().x
    Nchains = length(energies)
    for i in index:stride:Nchains

        # Inlining the Gaussian landscape calculation directly into the kernel
        μ1 = 0.0f0  # μ1 = (0,0) (zero vector)
        μ2 = 5.0f0  # μ2 = (5,5) (scalar for simplicity)
        
        s1 = 0.0f0 # squared difference for μ1
        s2 = 0.0f0 # squared difference for μ2
        
        # Compute squared differences directly in the kernel
        for j in 1:dim
            idx = (i-1)*dim + j
            s1 += (x[idx] - μ1)^2
            s2 += (x[idx] - μ2)^2
        end
        energy = exp(-0.5 * s1) + 0.5 * exp(-0.5 * s2)  # Combine the Gaussian terms
        energies[i] = energy

    end

    return
end

mock_energy_CUDA! (generic function with 1 method)

In [6]:
NTemps = 1000
test = zeros(Float32, NTemps)
x_tests = randn(Float32, NTemps, 2) # random initial guess
# D = size(x_tests, 2)
x_tests = vec(x_tests)
println(Array(test)')
println(Array(x_tests)')
test = CuArray(test)
x_tests = CuArray(x_tests);

Float32[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 

In [7]:
threads = 256
blocks = ceil(Int, NTemps / threads)
@cuda threads=threads blocks=blocks mock_energy_CUDA!(2, x_tests, test)

CUDA.KernelException: KernelException: exception thrown during kernel execution on device NVIDIA GeForce RTX 3070

In [28]:
energies = Array(test)
println(energies)
println("CUDA kernel executed successfully.")

CUDA.KernelException: KernelException: exception thrown during kernel execution on device NVIDIA GeForce RTX 3070

In [22]:
println(test)

[0.702069030979632, 0.00635038107578857, 0.9961445506202655, 0.361829947330668, 0.9447488076157026, 0.09016471225292985, 0.6897795126750456, 0.21504012264421304, 0.5006013026701498, 0.5749349807375022, 0.13347666428726884, 0.930202070580152, 0.9270573033923443, 0.16814577641998543, 0.16064463121712053, 0.3629347994661453, 0.03011052126267132, 0.9822010129644475, 0.16324206880639133, 0.7478333869834074, 0.8285441708929728, 0.2159129052918839, 0.4123264297915021, 0.7146490973682638, 0.7809971416034827, 0.6437493356839046, 0.10120395520481702, 0.036471006281305174, 0.06586711620934665, 0.48209361682963026, 0.03860025296482284, 0.3318381520942504, 0.4357001719586189, 0.6817803858027, 0.9703595248567374, 0.3726373635960034, 0.07915900278585089, 0.18115910256813572, 0.4459914140618134, 0.23076960088245915, 0.596864653558192, 0.7672291531399386, 0.15043479381108837, 0.225117463966015, 0.06866969137423372, 0.661797606876336, 0.7091568792276495, 0.5964677458976145, 0.16431669160701098, 0.719500

In [ ]:
function parallel_tempering(PTParameters::PTParams, MetropolisParams::MetropolisParams, initial_guess)

    # -------------------------------------------------------
    # Local variables
    # -------------------------------------------------------
    
    NTemps           = PTParameters.NTemps
    Nexchanges       = PTParameters.Nexchanges
    TMax             = PTParameters.TMax
    NSteps           = MetropolisParams.NSteps
    StepSize         = MetropolisParams.StepSize

    λ                = PTParameters.λ
    temperatures     = zeros(NTemps)
    temperatures[1]  = TMax
    [temperatures[i] = temperatures[i-1]*λ for i in 2:NTemps]

    xo               = initial_guess
    xn               = zeros(size(xo))
    D                = size(xo, 2)

    Eveco           = zeros(NTemps) # preallocate energy vector
    @cuda mock_energy_CUDA(xo, Eveco) # compute parallel energy
    TupEBest        = findmin(Eveco)
    EBest           = TupEBest[1]
    EBestPos        = xo[TupEBest[2]]

    display("Initial guess: ", xo)
    display("Initial energies: ", Eveco)
    display("Initial best energy: ", EBest[1])
    display("Temperatures: ", temperatures')
    # debug energy and init guess
    is_plot = false
    if is_plot
        clf()
        plot(-10:10, Ener.(-10:10), label="Energy")
        plot(xo, Ener.(xo), "ro", label="Initial guess")
        plot(EBestPos, EBest[1], "go", label="Best guess")
        title("Initial guess")
        xlabel("x")
        ylabel("Energy")
        legend()
        # if ./out exists, save the figure there
        if isdir("./out")
            savefig("./out/energy")
        else
            show()
        end
    end

    # -------------------------------------------------------
    # Parallel tempering loop
    # -------------------------------------------------------
    @showprogress for _ in 1:PTParameters.Nexchanges+1
        # Metropolis step
        for i in 1:NSteps
            xn = xo .+ StepSize*randn(NTemps, D) # generate new positions
            Evecn = zeros(NTemps) # preallocate energy vector
            @cuda threads=256 blocks=256 mock_energy_CUDA(xn, Evecn) # compute parallel energy
            ΔE_vec = Evecn .- Eveco # compute the energy difference between the new and old positions

            mask1 = ΔE_vec .< 0 # if the new position is better, accept it
            mask2 = !mask1 & (rand(NTemps) .< exp.(-ΔE_vec./temperatures)) # if the new position is worse, metropolis probability
            mask = mask1 .| mask2 # combine the two masks
            for i in 1:D
                xo[i] = xn[i] .* mask .+ xo[i] .* .!mask
            end

            Eveco = zeros(NTemps) # preallocate energy vector
            @cuda threads=256 blocks=256 mock_energy_CUDA(xo, Eveco) # compute parallel energy
            prob_best_guess = findmin(Eveco) # from the new sampled energies, find the best guess
            if prob_best_guess[1] < EBest # compare the new best guess with the old one. If its better, update it
                # EBest: energy, position
                # EBestPos: position
                EBest    = prob_best_guess[1]
                EBestPos = xo[prob_best_guess[2]]
            end
        end

        # Exchange step
        # no check for the lowest state, as we don't explore space here
        exchange_energies = zeros(NTemps)
        @cuda threads=256 blocks=256 mock_energy_CUDA(xo, exchange_energies) # compute parallel energy
        for temp in 1:NTemps-1
            ΔE_exchange_no_T = exchange_energies[temp] - exchange_energies[temp+1]
            ΔE_exchange = ΔE_exchange_no_T * (1/temperatures[temp] - 1/temperatures[temp+1])
            if ΔE_exchange < 0
                xo[temp, :], xo[temp+1, :] = xo[temp+1, :], xo[temp, :]
            elseif rand() < exp(-ΔE_exchange)
                xo[temp, :], xo[temp+1, :] = xo[temp+1, :], xo[temp, :]
            end
        end
    end

    return xo, EBest, EBestPos
end;

In [ ]:
function see_indices_in_gpu(indices)
    index = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    stride = blockDim().x * gridDim().x
    Nmax = size(indices, 1)

    # if index <= Nmax
        indices[index]=blockIdx().x
    # end
    nothing
end

indices = CUDA.zeros(257)

@cuda threads=256 blocks=256 see_indices_in_gpu(indices)

In [ ]:
indices_cpu = Array(indices)
println(indices_cpu)

In [ ]:
count = 0
for idx in indices_cpu
    if idx > 0
        count += 1
    end
end
println("Count: ", count)